# An investigation into possible colour choices that will not coincide under colour deficiency models

The ZX-calculus uses red and green nodes, and the community has been resistant to change in these colour choices.
Rather than try and change the colours of the calculus we aim to give standard shades of red and green,
which do not clash under the following colour models:

- Green-Blind/Deuteranopia
- Red-Blind/Protanopia
- Blue-Blind/Tritanopia
- Monochromacy/Achromatopsia (greyscale)

For the final result and instructions on how to implement these decisions in LaTeX see the bottom of this page.

## Method
We will use the python library colorspacious (DOI:10.5281/zenodo.1214904) to perform the colour manipulation,
and take advice on contrast from the W3 Standard (https://www.w3.org/TR/WCAG21/ .)
Any given pair of colours will be assessed on the following criteria:

- Contrast against black
- DeltaE difference between the colours (CAM02-UCS)

    
These contrasts will be evaluated for each transformed pair of colours.
The final candidates for the "red" and "green" will be found by starting with the web standard colours for those names,
then altering the saturation and lightness (by hand) keeping the original hues. The W3 standard requires a contrast difference greater that 4.5, but no standard is given for DeltaE values. As such we have chosen the value 5 as our DeltaE tolerance, but note that this is arbitrary and judged "good enough" by eye.

In [1]:
from colorspacious import cspace_convert
from colorspacious import cspace_converter
from colorspacious import deltaE as cspace_deltaE
from colour import Color
import ipywidgets as widgets
import math


In [2]:
# Implement colour space conversions

CVD_D_space = {"name": "sRGB1+CVD","cvd_type": "deuteranomaly","severity": 100}
CVD_P_space = {"name": "sRGB1+CVD","cvd_type": "protanomaly","severity": 100}
CVD_T_space = {"name": "sRGB1+CVD","cvd_type": "tritanomaly","severity": 100}

CVD_D = cspace_converter(CVD_D_space, "CAM02-UCS")
CVD_P = cspace_converter(CVD_P_space, "CAM02-UCS")
CVD_T = cspace_converter(CVD_T_space, "CAM02-UCS")


toCAM = cspace_converter("sRGB1", "CAM02-UCS")


def constrain(n):
    return min(max(math.floor(n),0),255)

toRGB_unconstrained = cspace_converter("CAM02-UCS", "sRGB1")

def toRGB(CAM_colour):
    return list(map(constrain, toRGB_unconstrained(CAM_colour)))

def grey(CAM_colour):
    jch = cspace_convert(CAM_colour, "CAM02-UCS", "JCh")
    jch[..., 1] = 0
    return cspace_convert(jch, "JCh", "CAM02-UCS")


def transformations(RGB_colour):
    return [
        CVD_P(RGB_colour),
        CVD_D(RGB_colour),
        CVD_T(RGB_colour),
        grey(toCAM(RGB_colour))
    ]

In [3]:

def render(CAM_colour, description = ''):
    RGB_colour = toRGB(CAM_colour)
    floor = math.floor
    display(widgets.ColorPicker(
    concise=False,
    description=description,
    value=('#%02x%02x%02x' % (floor(RGB_colour[0]),floor(RGB_colour[1]),floor(RGB_colour[2]))),
    disabled=True
    ))
    return RGB_colour

In [4]:
# Demonstrate renderer and transforms:

rgb = [200,100,50]
render(toCAM(rgb), "Original")
t = transformations(rgb)
print("CVD simulations:")
render(t[0], "Deuteranopia")
render(t[1], "Protanopia")
render(t[2], "Tritanopia")
render(t[3], "Greyscale")

ColorPicker(value='#c86432', description='Original', disabled=True)

CVD simulations:


ColorPicker(value='#80732a', description='Deuteranopia', disabled=True)

ColorPicker(value='#98892f', description='Protanopia', disabled=True)

ColorPicker(value='#da4d59', description='Tritanopia', disabled=True)

ColorPicker(value='#878484', description='Greyscale', disabled=True)

[135, 132, 132]

# Contrast
For the sRGB colorspace, the relative luminance of a color is defined as L = 0.2126 * R + 0.7152 * G + 0.0722 * B where R, G and B are defined as: 
- if RsRGB <= 0.03928 then R = RsRGB/12.92 else R = ((RsRGB+0.055)/1.055) ^ 2.4 
- if GsRGB <= 0.03928 then G = GsRGB/12.92 else G = ((GsRGB+0.055)/1.055) ^ 2.4 
- if BsRGB <= 0.03928 then B = BsRGB/12.92 else B = ((BsRGB+0.055)/1.055) ^ 2.4 

and RsRGB, GsRGB, and BsRGB are defined as:
- RsRGB = R8bit/255
- GsRGB = G8bit/255
- BsRGB = B8bit/255

In [5]:
def relative_luminance(CAM_colour):
    rgb_colour = toRGB(CAM_colour)
    RsRGB = rgb_colour[0] / 255
    GsRGB = rgb_colour[1] / 255
    BsRGB = rgb_colour[2] / 255
    def shift(n):
        if(n <= 0.03928):
            return n / 12.92
        else:
            return ((n+0.055)/1.055) ** 2.4 
    return 0.2126 * shift(RsRGB) + 0.7152 * shift(GsRGB) + 0.0722 * shift(BsRGB)

def contrast(CAM_colour1,CAM_colour2):
    proper = (relative_luminance(CAM_colour1) + 0.05) / (relative_luminance(CAM_colour2) + 0.05)
    return max(proper, 1.0 / proper)

def deltaE(CAM_colour1,CAM_colour2):
    return cspace_deltaE(CAM_colour1, CAM_colour2, input_space='CAM02-UCS', uniform_space='CAM02-UCS')

In [6]:
black = toCAM([0,0,0])
white = toCAM([255,255,255])

def calc_all(RGB1, RGB2):
    c1 = toCAM(RGB1)
    c2 = toCAM(RGB2)
    render(c1, "c1")
    render(c2, "c2")
    t1 = transformations(RGB1)
    t2 = transformations(RGB2)
    ts = list(zip(transformations(RGB1), transformations(RGB2)))
    print("Deuteranopia:")
    render(t1[0])
    render(t2[0])
    print("Protanopia:")
    render(t1[1])
    render(t2[1])
    print("Tritanopia:")
    render(t1[2])
    render(t2[2])
    print("Greyscale:")
    render(t1[3])
    render(t2[3])
    print("Lowest delta (non-greyscale:)", min(list(map(deltaE, t1[:3], t2[:3]))))
    print("Greyscale contrast:", contrast(t1[3],t2[3]))
    print("Black on top of c1 (lowest contrast:)", min(list(map(contrast,t1,[black]*4))))
    print("Black on top of c2 (lowest contrast:)", min(list(map(contrast,t2,[black]*4))))
    print("White behind c1 (lowest contrast:)", min(list(map(contrast,t1,[white]*4))))
    print("White behind c2 (lowest contrast:)", min(list(map(contrast,t2,[white]*4))))


In [7]:
input_col_1 = widgets.ColorPicker(
    concise=False,
    value=('#e8a6a6'),
    disabled=False
    )
input_col_2 = widgets.ColorPicker(
    concise=False,
    value=('#d8f8d8'),
    disabled=False
    )

print("Pick colours c1 and c2:")
display(input_col_1)
display(input_col_2)

def picker_to_rgb(p):
    return list(map(lambda x : constrain(x * 255), Color(p.value).rgb))

Pick colours c1 and c2:


ColorPicker(value='#e8a6a6')

ColorPicker(value='#d8f8d8')

In [8]:
print(picker_to_rgb(input_col_1), picker_to_rgb(input_col_2))
calc_all(picker_to_rgb(input_col_1), picker_to_rgb(input_col_2))

[232, 165, 165] [216, 248, 216]


ColorPicker(value='#e8a5a5', description='c1', disabled=True)

ColorPicker(value='#d8f8d8', description='c2', disabled=True)

Deuteranopia:


ColorPicker(value='#b1aea4', disabled=True)

...

ColorPicker(value='#d5f5ee', disabled=True)

Greyscale:


ColorPicker(value='#bcb8b8', disabled=True)

ColorPicker(value='#f1ecec', disabled=True)

Lowest delta (non-greyscale:) 6.469209178621033
Greyscale contrast: 1.6796313826031712
Black on top of c1 (lowest contrast:) 9.459910265257026
Black on top of c2 (lowest contrast:) 17.949561268460233
White behind c1 (lowest contrast:) 1.9258122856079662
White behind c2 (lowest contrast:) 1.1301822441445628


# How to implement in LaTeX?

Simply add the following two snippets of code to your preamble. The first defines the colours "zx_red" and "zx_green":

<pre>
\definecolor{zx_red}{RGB}{227, 145, 145}
\definecolor{zx_green}{RGB}{230, 250, 230}
</pre>

The second defines two node types "gn" and "rn" for use inside tikz diagrams:

<pre>
\tikzstyle{gn}=[rectangle,rounded corners=0.8em,fill=zx_green,draw=Black,line width=0.8 pt,inner sep=3pt,minimum width=1.5em,minimum height=1.5em]
\tikzstyle{rn}=[rectangle,rounded corners=0.8em,fill=zx_red,draw=Black,line width=0.8 pt,inner sep=3pt,minimum width=1.5em,minimum height=1.5em]
</pre>

Note that we encourage rounded rectangles over circles.